# 自定义层 定义后可以加入nn.Sequential



## 不带参数的层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到构建更复杂的模型中**]。


In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.3283e-09, grad_fn=<MeanBackward0>)

## **带参数的层**

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义序列化程序。

现在，让我们实现自定义版本的全连接层。
该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入和输出的数量。


In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [6]:
linear = MyLinear(5, 3)
print("weight: ",linear.weight)
print("bias: ",linear.bias)

weight:  Parameter containing:
tensor([[ 1.4751,  0.0840, -0.4861],
        [-0.1961,  1.5354, -0.7787],
        [-0.2362, -1.2956, -1.4082],
        [-1.3279,  1.3935,  0.6901],
        [ 0.7922, -0.4930,  1.7979]], requires_grad=True)
bias:  Parameter containing:
tensor([-0.1221, -0.2181, -0.7570], requires_grad=True)


我们可以[**使用自定义层直接执行前向传播计算**]。


In [7]:
linear(torch.rand(2, 5))

tensor([[0.8555, 0.0000, 0.0000],
        [0.6500, 0.8417, 0.0000]])

In [8]:
print(linear(torch.rand(2, 5)))
print(linear(torch.rand(3, 5)))

tensor([[0.0000, 0.7549, 0.0000],
        [0.3681, 0.0000, 0.0000]])
tensor([[0.8722, 0.0000, 0.0000],
        [0.5868, 0.5832, 0.0000],
        [0.0000, 0.7889, 0.0000]])


我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 0.0000],
        [14.9570]])